In [1]:
!pip install -U peft transformers accelerate datasets bitsandbytes -q


[notice] A new release of pip is available: 24.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip uninstall torch -y
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Found existing installation: torch 2.4.0+cu118Note: you may need to restart the kernel to use updated packages.

Uninstalling torch-2.4.0+cu118:
  Successfully uninstalled torch-2.4.0+cu118
Looking in indexes: https://download.pytorch.org/whl/cu118Note: you may need to restart the kernel to use updated packages.

  Using cached https://download.pytorch.org/whl/cu118/torch-2.4.0%2Bcu118-cp310-cp310-win_amd64.whl (2692.5 MB)


In [44]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
from sklearn.metrics import accuracy_score, f1_score
import os

In [ ]:
dataset = load_dataset("hendzh/PromptShield")
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

MAX_LEN = 256
BATCH_SIZE = 16
EPOCHS = 3

Using the latest cached version of the dataset since hendzh/PromptShield couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\incha\.cache\huggingface\datasets\hendzh___prompt_shield\default\0.0.0\a5234cb1f5cdb256600cab64b8c961195b5e8404 (last modified on Sat May 17 17:42:20 2025).


In [13]:
import transformers
print("Transformers version:", transformers.__version__)
print("Transformers path:", transformers.__file__)


Transformers version: 4.51.3
Transformers path: c:\Users\incha\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\__init__.py


In [15]:
%pip uninstall transformers -y
%pip install transformers==4.38.2 --force-reinstall

Found existing installation: transformers 4.51.3Note: you may need to restart the kernel to use updated packages.

Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-4.51.3

     ---------------------------------------- 0.0/130.7 kB ? eta -:--:--
     -------------------------------------- 130.7/130.7 kB 7.5 MB/s eta 0:00:00
  Using cached huggingface_hub-0.31.2-py3-none-any.whl.metadata (13 kB)
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 3.2 MB/s eta 0:00:00
  Using cached PyYAML-6.0.2-cp310-cp310-win_amd64.whl.metadata (2.1 kB)
  Using cached regex-2024.11.6-cp310-cp310-win_amd64.whl.metadata (41 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ---------------------------------------- 57.7/57.7 kB 3.0 MB/s eta 0:00:00
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metad

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awsebcli 3.20.10 requires colorama<0.4.4,>=0.2.5, but you have colorama 0.4.6 which is incompatible.
awsebcli 3.20.10 requires urllib3<2,>=1.26.5, but you have urllib3 2.4.0 which is incompatible.
botocore 1.31.85 requires urllib3<2.1,>=1.25.4; python_version >= "3.10", but you have urllib3 2.4.0 which is incompatible.
chromadb 0.5.5 requires numpy<2.0.0,>=1.22.5, but you have numpy 2.2.5 which is incompatible.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
faiss-cpu 1.8.0.post1 requires numpy<2.0,>=1.0, but you have numpy 2.2.5 whic

In [31]:
def tokenize_fn(example):
    return tokenizer(
        example["prompt"], 
        padding="max_length", 
        truncation=True, 
        max_length=MAX_LEN
    )

tokenized = dataset.map(tokenize_fn, batched=True)
train_dataset = tokenized["train"].with_format("torch")
val_dataset = tokenized["validation"].with_format("torch")

# ✅ Load base model with quantization (4-bit)
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

base_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    quantization_config=bnb_config,
    device_map="auto"
)

Map:   0%|          | 0/18909 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/23516 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
torch.cuda.is_available()

True

In [32]:
base_model = prepare_model_for_kbit_training(base_model)


In [9]:
import torch
print("Is CUDA available?", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())
print("CUDA device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "N/A")


Is CUDA available? True
CUDA device count: 1
CUDA device name: NVIDIA GeForce GTX 1650


In [36]:
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    # Correct target modules for BERT-base
    target_modules=[
        "bert.encoder.layer.0.attention.self.query",
        "bert.encoder.layer.0.attention.self.key",
        "bert.encoder.layer.0.attention.self.value",
        "bert.encoder.layer.0.attention.output.dense",
        "bert.encoder.layer.0.intermediate.dense",
        "bert.encoder.layer.0.output.dense",
        # Add more layers as needed
    ]
)


In [37]:
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

trainable params: 112,130 || all params: 109,595,908 || trainable%: 0.1023


In [39]:
import os
output_dir = "./qlora-bert-base"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="./qlora-bert-base",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    learning_rate=2e-4,
    load_best_model_at_end=True,
    report_to="none",
    logging_dir="./logs"
)

TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [21]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), dim=1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

C:\Users\incha\AppData\Local\Temp\ipykernel_21372\2721217396.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [23]:
trainer.train()

c:\Users\incha\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\distilbert\modeling_distilbert.py:401: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
c:\Users\incha\AppData\Local\Programs\Python\Python310\lib\site-packages\bitsandbytes\nn\modules.py:354: UserWarning: FP4 quantization state not initialized. Please call .cuda() or .to(device) on the LinearFP4 layer first.
  warnings.warn(


AssertionError: 

In [ ]:
model.save_pretrained("./qlora-distilbert")
tokenizer.save_pretrained("./qlora-distilbert")


In [34]:
import torch
from transformers import AutoModelForSequenceClassification
from peft import prepare_model_for_kbit_training
from transformers import BitsAndBytesConfig

# Load the model with the same configuration
model_name = "bert-base-uncased"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

base_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    quantization_config=bnb_config,
    device_map="auto"
)

base_model = prepare_model_for_kbit_training(base_model)

# Print all module names to identify the correct target modules
print("=== MODULES IN BERT MODEL ===")
for name, module in base_model.named_modules():
    if isinstance(module, torch.nn.Linear):
        print(f"Linear module: {name}, shape: {module.weight.shape}")
    # Also print attention modules
    elif "attention" in name.lower():
        print(f"Attention module: {name}")

print("\n=== RECOMMENDED TARGET MODULES ===")
# Find potential modules that could be targeted for LoRA
linear_modules = [name for name, module in base_model.named_modules() 
                 if isinstance(module, torch.nn.Linear)]
print("Potential target modules (Linear layers):", linear_modules)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


=== MODULES IN BERT MODEL ===
Attention module: bert.encoder.layer.0.attention
Attention module: bert.encoder.layer.0.attention.self
Linear module: bert.encoder.layer.0.attention.self.query, shape: torch.Size([294912, 1])
Linear module: bert.encoder.layer.0.attention.self.key, shape: torch.Size([294912, 1])
Linear module: bert.encoder.layer.0.attention.self.value, shape: torch.Size([294912, 1])
Attention module: bert.encoder.layer.0.attention.self.dropout
Attention module: bert.encoder.layer.0.attention.output
Linear module: bert.encoder.layer.0.attention.output.dense, shape: torch.Size([294912, 1])
Attention module: bert.encoder.layer.0.attention.output.LayerNorm
Attention module: bert.encoder.layer.0.attention.output.dropout
Linear module: bert.encoder.layer.0.intermediate.dense, shape: torch.Size([1179648, 1])
Linear module: bert.encoder.layer.0.output.dense, shape: torch.Size([1179648, 1])
Attention module: bert.encoder.layer.1.attention
Attention module: bert.encoder.layer.1.atten